In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pytorch-tabnet transformers

import pandas as pd
import shutil
import os
import numpy as np
from transformers import BertTokenizer
from transformers import BertModel
import torch
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns

Mounted at /content/drive
     |████████████████████████████████| 1.5MB 17.4MB/s 
     |████████████████████████████████| 890kB 55.6MB/s 
     |████████████████████████████████| 2.9MB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ce217e642ed0d37ed2b2d659974620f2e87f5dc60cad08f526388f1c77997d17
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/test.csv', sep=';')
ed = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/education.csv', sep=';')
ep = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/employements.csv', sep=';')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/test.csv', sep=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131259 entries, 0 to 131258
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          131259 non-null  int64  
 1   position                    131259 non-null  object 
 2   region                      131254 non-null  object 
 3   industry                    131259 non-null  object 
 4   locality                    131259 non-null  int64  
 5   locality_name               124590 non-null  object 
 6   education_type              113833 non-null  object 
 7   drive_licences              52356 non-null   object 
 8   citizenship                 131259 non-null  object 
 9   schedule                    131259 non-null  object 
 10  employement_type            131259 non-null  object 
 11  age                         127962 non-null  float64
 12  gender                      127967 non-null  object 
 13  experience    

In [4]:
df_salary = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/salary_by_city - salary_by_city - Лист1.csv')
df_salary['locality_name'] = df_salary['location']
df_salary['salary_count'] = df_salary['salary']
df_1 = pd.merge(df, df_salary, how='left', on="locality_name")

In [5]:
df_1.salary_count.fillna('28500')

0          34000
1          28500
2          28500
3          28500
4          28500
           ...  
131254     28500
131255     28500
131256    103100
131257     35900
131258     28500
Name: salary_count, Length: 131259, dtype: object

In [6]:
df_1['salary_count'].isna().sum()

74264

In [7]:
sd_median=df_1['salary_count'].median(skipna = bool)
df_1['salary_count']= pd.to_numeric(df_1['salary_count'],errors='coerce')
df_1['salary_count']= np.where(np.isnan(pd.to_numeric(df_1['salary_count'],errors='coerce')),sd_median,df_1['salary_count'])

In [8]:
sd_median=df_1['salary_count'].median(skipna = bool)
df_1['salary_count']= pd.to_numeric(df_1['salary_count'],errors='coerce')
df_1['salary_count']= np.where(np.isnan(pd.to_numeric(df_1['salary_count'],errors='coerce')),sd_median,df_1['salary_count'])

In [9]:
df_1['salary_count'].isna().sum()

0

In [10]:
from sklearn.preprocessing import StandardScaler
import numpy as np
X=df_1.iloc[:, 26].values.reshape(-1, 1)
X=np.nan_to_num(X)
Clus_dataSet=StandardScaler().fit_transform(X)

In [11]:
X=df_1.iloc[:, 26].values
print(X)

[ 34000.  43600.  43600. ... 103100.  35900.  43600.]


In [12]:
from sklearn.cluster import KMeans
clasterNum=10
k_maens = KMeans(init="k-means++",n_clusters=clasterNum,n_init=12)
k_maens.fit(Clus_dataSet)
labels=k_maens.labels_

In [13]:
df_1['cluster'] = labels

In [14]:
df_1 = pd.merge(df_1, ep, how='left', on="id")

In [15]:
df_1.responsibilities = df_1.responsibilities.str.replace(r'\(.*?\)', '')
df_1.responsibilities = df_1.responsibilities.str.replace('/', '')
df_1.responsibilities = df_1.responsibilities.str.replace(',', '')
df_1.responsibilities = df_1.responsibilities.str.replace(' +', ' ')
df_1.responsibilities = df_1.responsibilities.str.strip()

df_1['responsibilities']= np.where( pd.isnull(df_1['responsibilities']),'no responsibilities',df_1['responsibilities']) 
df_1['responsibilities'].isna().sum()

0

In [16]:
df_1['education_type']= np.where( pd.isnull(df_1['education_type']),'no education type',df_1['education_type']) 
df_1['education_type'].isna().sum()

0

In [17]:
df_work = df_1[['id', 'industry', 'employement_type',
              'education_type', 'cluster', 'responsibilities']]

In [18]:
df_1.responsibilities = df_1.responsibilities.str.replace(r'<(.*?)>', '')

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [20]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

In [21]:
#df_work[['responsibilities']] = df_work[['responsibilities']].astype(str)
df_work.loc[:, ('responsibilities')] = df_work.loc[:, ('responsibilities')].astype(str)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [22]:
%%time

bert.to(device)
bert.eval();

step = 10_000
responsibilities_vectors = False

for batch_i in range(0, len(df_work), step):
    batch = df_work.responsibilities[batch_i:batch_i+step]
    inputs = tokenizer(
        batch.to_list(),
        padding='max_length',
        truncation=True,
        max_length=20,
        return_tensors='pt',
        return_attention_mask=True,
    )
    inputs.to(device)
    with torch.no_grad():
        output = bert(**inputs)
    total_sum = torch.zeros_like(output[0][:, 1, :])
    for i in range(0, output[0].shape[1]):
        total_sum += output[0][:, i, :]
    total_sum = total_sum / output[0].shape[1]
    # responsibilities_vectors.extend(total_sum.detach().cpu().numpy())
    total_sum = total_sum.detach().cpu().numpy()
    if responsibilities_vectors is False:
        responsibilities_vectors = total_sum
    else:
        responsibilities_vectors = np.vstack((responsibilities_vectors, total_sum))
    
    if batch_i % 100_000 == 0:
        print(batch_i)

0
100000
CPU times: user 3min 10s, sys: 1min 15s, total: 4min 26s
Wall time: 4min 35s


In [23]:
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/responsibilities_vectors.npy', responsibilities_vectors)

In [50]:
df_work.to_csv('df_work_test.csv')

In [24]:
cluster = set()
education_type = set()
employement_type = set()
industry = set()

cluster.update(df_work.cluster.unique().tolist())
education_type.update(df_work.education_type.unique().tolist())
employement_type.update(df_work.employement_type.unique().tolist())
industry.update(df_work.industry.unique().tolist())

In [25]:
cluster2id = {name:id for id, name in enumerate(cluster)}
education_type2id = {name:id for id, name in enumerate(education_type)}
employement_type2id = {name:id for id, name in enumerate(employement_type)}
industry2id = {name:id for id, name in enumerate(industry)}

In [26]:
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/cluster2id.pickle', 'wb') as handle:
    pickle.dump(cluster2id, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/education_type2id.pickle', 'wb') as handle:
    pickle.dump(education_type2id, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/employement_type2id.pickle', 'wb') as handle:
    pickle.dump(employement_type2id, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/industry2id.pickle', 'wb') as handle:
    pickle.dump(industry2id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
cluster_label = []
education_type_label = []
employement_type_label = []
industry_label = []


for index in df_work.index:
    cluster_label.append(cluster2id[df_work.cluster.iloc[index]])
    education_type_label.append(education_type2id[df_work.education_type.iloc[index]])
    employement_type_label.append(employement_type2id[df_work.employement_type.iloc[index]])
    industry_label.append(industry2id[df_work.industry.iloc[index]])
    

In [28]:
cluster_label = np.array(cluster_label)
education_type_label = np.array(education_type_label)
employement_type_label = np.array(employement_type_label)
industry_label = np.array(industry_label)

In [29]:
cluster_label = np.reshape(cluster_label, (cluster_label.shape[0], 1))
education_type_label = np.reshape(education_type_label, (education_type_label.shape[0], 1))
employement_type_label = np.reshape(employement_type_label, (employement_type_label.shape[0], 1))
industry_label = np.reshape(industry_label, (industry_label.shape[0], 1))

In [30]:

np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/cluster_label.npy', cluster_label)
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/education_type_label.npy', education_type_label)
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/employement_type_label.npy', employement_type_label)
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/industry_label.npy', industry_label)

In [42]:
df_work.columns[0]


'id'

In [47]:
id_array = df[['id']].to_numpy()
id_array

array([[     2],
       [     6],
       [     7],
       ...,
       [437526],
       [437527],
       [437528]])

In [48]:
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/id_array.npy', id_array)
